### Context of this Notebook
Define a RAG workflow for generating our CSV from a query using LangGraph.

#### Design

The purpose is to take our pdf and generate a resulting csv with as much accuracy towards our intended purpose.
The overarching framework will be defined by LangGraph. Within the LangGraph, we need a couple things.

**State:** As the workflow is running, a common state object will be passed around describing the status quo of the LangGraph.

**Agents:** The workflow will be run by several agents. In context of what we are doing, there are several that we might need.

In [6]:
# Document we are working with
file_name = "sample-new-fidelity-acnt-stmt"
local_llm = "llama3.1:70b" # llama3 for 8b

# Run OCR on PDF and gives us text file
%run -i pdf_to_text.py $file_name

In [3]:
# RETREIVER DEFINITION
import os
from langchain_community.vectorstores import faiss
from langchain_nomic.embeddings import NomicEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# if not os.path.exists(f"{file_name}"):
#     os.makedirs("text_data")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024,
    chunk_overlap=200,
)
embedding = NomicEmbeddings(model="nomic-embed-text-v1.5", inference_mode="local")
os.makedirs("vectorstore", exist_ok=True)
if not os.path.exists(f"vectorstore/{file_name}.faiss"):
    with open(f"text_data/{file_name}.txt", "r") as f:
        pdf_text = f.read()
        passages = text_splitter.split_text(pdf_text)
        # Define open source embeddings
        vectorstore = faiss.FAISS.from_texts(
            texts=passages,
            embedding=embedding,
        )
        vectorstore.save_local("vectorstore",file_name)
else:
    vectorstore = faiss.FAISS.load_local("vectorstore",embedding, file_name, allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever(search_kwargs={"k": 20})
retriever.invoke("Quantity of stocks and securities for the purpose of underwriting.")

[Document(page_content="Advisers, Inc. (SAI) are carried by NFS and covered by SIPC but do not contribute to your margin and maintenance requirements.  Short Account Balances Securities sold short are held in a segregated short account. These securities are marked-to-market for margin purposes, and any increase or decrease from the previous week's value is transferred weekly to your margin account. Fidelity represents your short account balance as of the last weekly mark-to-market, not as of the statement end date.  Information About Your Option Transactions Each transaction confirmation previously delivered to you contains full information about commissions and other charges, and such information is available promptly upon request. Assignments of American and European-style options are allocated among customer short positions pursuant to a random allocation procedure, a description is available upon request. Short positions in American-style options are liable for assignment anytime. 

In [7]:
### RETRIEVAL GRADER (MINIMUM TEST)

from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
# LLM
llm = ChatOllama(model=local_llm, format="json", temperature=0)

prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing relevance 
    of a retrieved document to a user question. If the document contains keywords related to the user question, 
    grade it as relevant. It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
    Provide the binary score as a JSON with a single key 'score' and no premable or explanation.
     <|eot_id|><|start_header_id|>user<|end_header_id|>
    Here is the retrieved document: \n\n {document} \n\n
    Here is the user question: {question} \n <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["question", "document"],
)

retrieval_grader = prompt | llm | JsonOutputParser()
question = "agent memory"
docs = retriever.invoke(question)
doc_txt = docs[1].page_content
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

{'score': 'no'}


In [18]:
import langgraph

from typing import Annotated, Sequence, TypedDict
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages

class AgentState(TypedDict):
    # The add_messages function defines how an update should be processed
    # Default is to replace. add_messages says "append"
    messages: Annotated[Sequence[BaseMessage], add_messages]

def retrieval(state) -> Sequence[langgraph.Document]:
    last_message = state["messages"][-1]
    state["documents"] = retriever.invoke(last_message.text)
    return 